# Generate Binary Threshold

In [ ]:
import matplotlib.pylab as plt
import cv2
import numpy as np

global first_frame
global filter_flag
first_frame = 1

def binary_threshold(img, low, high):
    output = np.zeros_like(img[:,:,0])
    mask = (img[:,:,0] >= low[0]) & (img[:,:,0] <= high[0]) \
        & (img[:,:,1] >= low[1]) & (img[:,:,1] <= high[1]) \
        & (img[:,:,2] >= low[2]) & (img[:,:,2] <= high[2])
            
    output[mask] = 1
    return output
